In [1]:
import os
import numpy as np
from PIL import Image
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from tqdm import tqdm
import pandas as pd

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
import torchvision
from torch.autograd import Variable
import matplotlib.pyplot as plt
from functions import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
import pickle


In [9]:
## ------------------- label conversion tools ------------------ ##
def labels2cat(label_encoder, list):
    return label_encoder.transform(list)

def labels2onehot(OneHotEncoder, label_encoder, list):
    return OneHotEncoder.transform(label_encoder.transform(list).reshape(-1, 1)).toarray()

def onehot2labels(label_encoder, y_onehot):
    return label_encoder.inverse_transform(np.where(y_onehot == 1)[1]).tolist()

def cat2labels(label_encoder, y_cat):
    return label_encoder.inverse_transform(y_cat).tolist()




# for CRNN
class Dataset_CRNN(data.Dataset):
    "Characterizes a dataset for PyTorch"
    def __init__(self, data_path, folders, labels, frames, transform=None):
        "Initialization"
        self.data_path = data_path
        self.labels = labels
        self.folders = folders
        self.transform = transform
        self.frames = frames

    def __len__(self):
        "Denotes the total number of samples"
        return len(self.folders)

    def read_images(self, path, selected_folder, use_transform):
        X = []
        for i in self.frames:
            image = Image.open(os.path.join(path, selected_folder, '{:05d}.jpg'.format(i)))

            if use_transform is not None:
                image = use_transform(image)

            X.append(image)
        X = torch.stack(X, dim=0)

        return X

    def __getitem__(self, index):
        "Generates one sample of data"
        # Select sample
        folder = self.folders[index]

        # Load data
        X = self.read_images(self.data_path, folder, self.transform)     # (input) spatial images
        y = torch.LongTensor([self.labels[index]])                  # (labels) LongTensor are for int64 instead of FloatTensor

        # print(X.shape)
        return X, y

## ---------------------- end of Dataloaders ---------------------- ##



## -------------------- (reload) model prediction ---------------------- ##
def Conv3d_final_prediction(model, device, loader):
    model.eval()

    all_y_pred = []
    with torch.no_grad():
        for batch_idx, (X, y) in enumerate(tqdm(loader)):
            # distribute data to device
            X = X.to(device)
            output = model(X)
            y_pred = output.max(1, keepdim=True)[1]  # location of max log-probability as prediction
            all_y_pred.extend(y_pred.cpu().data.squeeze().numpy().tolist())

    return all_y_pred


def CRNN_final_prediction(model, device, loader):
    cnn_encoder, rnn_decoder = model
    cnn_encoder.eval()
    rnn_decoder.eval()

    all_y_pred = []
    with torch.no_grad():
        for batch_idx, (X, y) in enumerate(tqdm(loader)):
            # distribute data to device
            X = X.to(device)
            output = rnn_decoder(cnn_encoder(X))
            y_pred = output.max(1, keepdim=True)[1]  # location of max log-probability as prediction
            all_y_pred.extend(y_pred.cpu().data.squeeze().numpy().tolist())

    return all_y_pred

## -------------------- end of model prediction ---------------------- ##



## ------------------------ 3D CNN module ---------------------- ##
def conv3D_output_size(img_size, padding, kernel_size, stride):
    # compute output shape of conv3D
    outshape = (np.floor((img_size[0] + 2 * padding[0] - (kernel_size[0] - 1) - 1) / stride[0] + 1).astype(int),
                np.floor((img_size[1] + 2 * padding[1] - (kernel_size[1] - 1) - 1) / stride[1] + 1).astype(int),
                np.floor((img_size[2] + 2 * padding[2] - (kernel_size[2] - 1) - 1) / stride[2] + 1).astype(int))
    return outshape

class CNN3D(nn.Module):
    def __init__(self, t_dim=120, img_x=90, img_y=120, drop_p=0.2, fc_hidden1=256, fc_hidden2=128, num_classes=50):
        super(CNN3D, self).__init__()

        # set video dimension
        self.t_dim = t_dim
        self.img_x = img_x
        self.img_y = img_y
        # fully connected layer hidden nodes
        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p
        self.num_classes = num_classes
        self.ch1, self.ch2 = 32, 48
        self.k1, self.k2 = (5, 5, 5), (3, 3, 3)  # 3d kernel size
        self.s1, self.s2 = (2, 2, 2), (2, 2, 2)  # 3d strides
        self.pd1, self.pd2 = (0, 0, 0), (0, 0, 0)  # 3d padding

        # compute conv1 & conv2 output shape
        self.conv1_outshape = conv3D_output_size((self.t_dim, self.img_x, self.img_y), self.pd1, self.k1, self.s1)
        self.conv2_outshape = conv3D_output_size(self.conv1_outshape, self.pd2, self.k2, self.s2)

        self.conv1 = nn.Conv3d(in_channels=1, out_channels=self.ch1, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.bn1 = nn.BatchNorm3d(self.ch1)
        self.conv2 = nn.Conv3d(in_channels=self.ch1, out_channels=self.ch2, kernel_size=self.k2, stride=self.s2,
                               padding=self.pd2)
        self.bn2 = nn.BatchNorm3d(self.ch2)
        self.relu = nn.ReLU(inplace=True)
        self.drop = nn.Dropout3d(self.drop_p)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(self.ch2 * self.conv2_outshape[0] * self.conv2_outshape[1] * self.conv2_outshape[2],
                             self.fc_hidden1)  # fully connected hidden layer
        self.fc2 = nn.Linear(self.fc_hidden1, self.fc_hidden2)
        self.fc3 = nn.Linear(self.fc_hidden2, self.num_classes)  # fully connected layer, output = multi-classes

    def forward(self, x_3d):
        # Conv 1
        x = self.conv1(x_3d)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        # Conv 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.drop(x)
        # FC 1 and 2
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc3(x)

        return x

## --------------------- end of 3D CNN module ---------------- ##



## ------------------------ CRNN module ---------------------- ##

def conv2D_output_size(img_size, padding, kernel_size, stride):
    # compute output shape of conv2D
    outshape = (np.floor((img_size[0] + 2 * padding[0] - (kernel_size[0] - 1) - 1) / stride[0] + 1).astype(int),
                np.floor((img_size[1] + 2 * padding[1] - (kernel_size[1] - 1) - 1) / stride[1] + 1).astype(int))
    return outshape


# 2D CNN encoder train from scratch (no transfer learning)
class EncoderCNN(nn.Module):
    def __init__(self, img_x=90, img_y=120, fc_hidden1=512, fc_hidden2=512, drop_p=0.3, CNN_embed_dim=300):
        super(EncoderCNN, self).__init__()

        self.img_x = img_x
        self.img_y = img_y
        self.CNN_embed_dim = CNN_embed_dim

        # CNN architechtures
        self.ch1, self.ch2, self.ch3, self.ch4 = 32, 64, 128, 256
        self.k1, self.k2, self.k3, self.k4 = (5, 5), (3, 3), (3, 3), (3, 3)      # 2d kernal size
        self.s1, self.s2, self.s3, self.s4 = (2, 2), (2, 2), (2, 2), (2, 2)      # 2d strides
        self.pd1, self.pd2, self.pd3, self.pd4 = (0, 0), (0, 0), (0, 0), (0, 0)  # 2d padding

        # conv2D output shapes
        self.conv1_outshape = conv2D_output_size((self.img_x, self.img_y), self.pd1, self.k1, self.s1)  # Conv1 output shape
        self.conv2_outshape = conv2D_output_size(self.conv1_outshape, self.pd2, self.k2, self.s2)
        self.conv3_outshape = conv2D_output_size(self.conv2_outshape, self.pd3, self.k3, self.s3)
        self.conv4_outshape = conv2D_output_size(self.conv3_outshape, self.pd4, self.k4, self.s4)

        # fully connected layer hidden nodes
        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=self.ch1, kernel_size=self.k1, stride=self.s1, padding=self.pd1),
            nn.BatchNorm2d(self.ch1, momentum=0.01),
            nn.ReLU(inplace=True),                      
            # nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=self.ch1, out_channels=self.ch2, kernel_size=self.k2, stride=self.s2, padding=self.pd2),
            nn.BatchNorm2d(self.ch2, momentum=0.01),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=2),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=self.ch2, out_channels=self.ch3, kernel_size=self.k3, stride=self.s3, padding=self.pd3),
            nn.BatchNorm2d(self.ch3, momentum=0.01),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=2),
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=self.ch3, out_channels=self.ch4, kernel_size=self.k4, stride=self.s4, padding=self.pd4),
            nn.BatchNorm2d(self.ch4, momentum=0.01),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=2),
        )

        self.drop = nn.Dropout2d(self.drop_p)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(self.ch4 * self.conv4_outshape[0] * self.conv4_outshape[1], self.fc_hidden1)   # fully connected layer, output k classes
        self.fc2 = nn.Linear(self.fc_hidden1, self.fc_hidden2)
        self.fc3 = nn.Linear(self.fc_hidden2, self.CNN_embed_dim)   # output = CNN embedding latent variables

    def forward(self, x_3d):
        cnn_embed_seq = []
        for t in range(x_3d.size(1)):
            # CNNs
            x = self.conv1(x_3d[:, t, :, :, :])
            x = self.conv2(x)
            x = self.conv3(x)
            x = self.conv4(x)
            x = x.view(x.size(0), -1)           # flatten the output of conv

            # FC layers
            x = F.relu(self.fc1(x))
            # x = F.dropout(x, p=self.drop_p, training=self.training)
            x = F.relu(self.fc2(x))
            x = F.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)
            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        # cnn_embed_seq: shape=(batch, time_step, input_size)

        return cnn_embed_seq


# 2D CNN encoder using ResNet-152 pretrained
class ResCNNEncoder(nn.Module):
    def __init__(self, fc_hidden1=512, fc_hidden2=512, drop_p=0.3, CNN_embed_dim=300):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(ResCNNEncoder, self).__init__()

        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p

        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-1]      # delete the last fc layer.
        self.resnet = nn.Sequential(*modules)
        self.fc1 = nn.Linear(resnet.fc.in_features, fc_hidden1)
        self.bn1 = nn.BatchNorm1d(fc_hidden1, momentum=0.01)
        self.fc2 = nn.Linear(fc_hidden1, fc_hidden2)
        self.bn2 = nn.BatchNorm1d(fc_hidden2, momentum=0.01)
        self.fc3 = nn.Linear(fc_hidden2, CNN_embed_dim)
        
    def forward(self, x_3d):
        cnn_embed_seq = []
        for t in range(x_3d.size(1)):
            # ResNet CNN
            with torch.no_grad():
                x = self.resnet(x_3d[:, t, :, :, :])  # ResNet
                x = x.view(x.size(0), -1)             # flatten output of conv

            # FC layers
            x = self.bn1(self.fc1(x))
            x = F.relu(x)
            x = self.bn2(self.fc2(x))
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)

            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)
        # cnn_embed_seq: shape=(batch, time_step, input_size)

        return cnn_embed_seq


class DecoderRNN(nn.Module):
    def __init__(self, CNN_embed_dim=300, h_RNN_layers=3, h_RNN=256, h_FC_dim=128, drop_p=0.3, num_classes=50):
        super(DecoderRNN, self).__init__()

        self.RNN_input_size = CNN_embed_dim
        self.h_RNN_layers = h_RNN_layers   # RNN hidden layers
        self.h_RNN = h_RNN                 # RNN hidden nodes
        self.h_FC_dim = h_FC_dim
        self.drop_p = drop_p
        self.num_classes = num_classes

        self.LSTM = nn.LSTM(
            input_size=self.RNN_input_size,
            hidden_size=self.h_RNN,        
            num_layers=h_RNN_layers,       
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.fc1 = nn.Linear(self.h_RNN, self.h_FC_dim)
        self.fc2 = nn.Linear(self.h_FC_dim, self.num_classes)

    def forward(self, x_RNN):
        
        self.LSTM.flatten_parameters()
        RNN_out, (h_n, h_c) = self.LSTM(x_RNN, None)  
        """ h_n shape (n_layers, batch, hidden_size), h_c shape (n_layers, batch, hidden_size) """ 
        """ None represents zero initial hidden state. RNN_out has shape=(batch, time_step, output_size) """

        # FC layers
        x = self.fc1(RNN_out[:, -1, :])   # choose RNN_out at the last time step
        x = F.relu(x)
        x = F.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc2(x)

        return x

## ---------------------- end of CRNN module ---------------------- ##


In [10]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    # set model as training mode
    cnn_encoder, rnn_decoder = model
    cnn_encoder.train()
    rnn_decoder.train()

    losses = []
    scores = []
    N_count = 0   # counting total trained sample in one epoch
    for batch_idx, (X, y) in enumerate(train_loader):
        # distribute data to device
        X, y = X.to(device), y.to(device).view(-1, )

        N_count += X.size(0)

        optimizer.zero_grad()
        output = rnn_decoder(cnn_encoder(X))   # output has dim = (batch, number of classes)

        loss = F.cross_entropy(output, y)
        losses.append(loss.item())

        # to compute accuracy
        y_pred = torch.max(output, 1)[1]  # y_pred != output
        step_score = accuracy_score(y.cpu().data.squeeze().numpy(), y_pred.cpu().data.squeeze().numpy())
        scores.append(step_score)         # computed on CPU

        loss.backward()
        optimizer.step()

        # show information
        if (batch_idx + 1) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accu: {:.2f}%'.format(
                epoch + 1, N_count, len(train_loader.dataset), 100. * (batch_idx + 1) / len(train_loader), loss.item(), 100 * step_score))

    return losses, scores

In [11]:
def validation(model, device, optimizer, test_loader, epoch):
    # set model as testing mode
    cnn_encoder, rnn_decoder = model
    cnn_encoder.eval()
    rnn_decoder.eval()

    test_loss = 0
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        for X, y in test_loader:
            # distribute data to device
            X, y = X.to(device), y.to(device).view(-1, )

            output = rnn_decoder(cnn_encoder(X))

            loss = F.cross_entropy(output, y, reduction='sum')
            test_loss += loss.item()                 # sum up batch loss
            y_pred = output.max(1, keepdim=True)[1]  # (y_pred != output) get the index of the max log-probability

            # collect all y and y_pred in all batches
            all_y.extend(y)
            all_y_pred.extend(y_pred)

    test_loss /= len(test_loader.dataset)

    # compute accuracy
    all_y = torch.stack(all_y, dim=0)
    all_y_pred = torch.stack(all_y_pred, dim=0)
    test_score = accuracy_score(all_y.cpu().data.squeeze().numpy(), all_y_pred.cpu().data.squeeze().numpy())

    # show information
    print('\nTest set ({:d} samples): Average loss: {:.4f}, Accuracy: {:.2f}%\n'.format(len(all_y), test_loss, 100* test_score))

    # save Pytorch models of best record
    torch.save(cnn_encoder.state_dict(), os.path.join(save_model_path, 'cnn_encoder_epoch{}.pth'.format(epoch + 1)))  # save spatial_encoder
    torch.save(rnn_decoder.state_dict(), os.path.join(save_model_path, 'rnn_decoder_epoch{}.pth'.format(epoch + 1)))  # save motion_encoder
    torch.save(optimizer.state_dict(), os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(epoch + 1)))      # save optimizer
    print("Epoch {} model saved!".format(epoch + 1))

    return test_loss, test_score

In [12]:
# set path
data_path = "F:/STAT453-Project/sub_dataset"    # define UCF-101 RGB data path

# action_name_path = './UCF101actions.pkl'
save_model_path = "./CRNN_ckpt/"

# EncoderCNN architecture
CNN_fc_hidden1, CNN_fc_hidden2 = 1024, 768
CNN_embed_dim = 512      # latent dim extracted by 2D CNN
img_x, img_y = 100, 176  # resize video 2d frame size
dropout_p = 0.0          # dropout probability

# DecoderRNN architecture
RNN_hidden_layers = 3
RNN_hidden_nodes = 512
RNN_FC_dim = 256

# training parameters
k = 10             # number of target category
epochs = 120        # training epochs
batch_size = 30  
learning_rate = 1e-4
log_interval = 10   # interval for displaying training info

# Select which frame to begin & end in videos
begin_frame, end_frame, skip_frame = 1, 10, 1


In [25]:
def main():
    # Detect devices
    use_cuda = torch.cuda.is_available()                   # check if GPU exists
    device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

    # Data loading parameters
    params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 0, 'pin_memory': True} if use_cuda else {}

    # load actions names
    action_names = pd.read_csv('labels.csv')['Label'].tolist()

    # convert labels -> category
    le = LabelEncoder()
    le.fit(action_names)

    # show how many classes there are
    list(le.classes_)

    # convert category -> 1-hot
    action_category = le.transform(action_names).reshape(-1, 1)
    enc = OneHotEncoder()
    enc.fit(action_category)

    # # example
    # y = ['HorseRace', 'YoYo', 'WalkingWithDog']
    # y_onehot = labels2onehot(enc, le, y)
    # y2 = onehot2labels(le, y_onehot)


#     # TODO read train.csv
#     actions = []
#     fnames = os.listdir(data_path)

#     all_names = []
#     for f in fnames:
#         loc1 = f.find('v_')
#         loc2 = f.find('_g')
#         actions.append(f[(loc1 + 2): loc2])

#         all_names.append(f)


#     # list all data files
#     all_X_list = all_names                  # all video file names
#     all_y_list = labels2cat(le, actions)    # all video labels

#     # train, test split
#     # TODO
#     train_list, test_list, train_label, test_label = train_test_split(all_X_list, all_y_list, test_size=0.25, random_state=42)
    
    train_df = pd.read_csv('train_set.csv')
    
    train_list = list(map(str, train_df['Index'].tolist()))
    train_label = labels2cat(le, train_df['Label'].tolist())
    
    test_df = pd.read_csv('val_set.csv')
    test_list = list(map(str, test_df['Index'].tolist()))
    test_label = labels2cat(le, test_df['Label'].tolist())

    transform = transforms.Compose([transforms.Resize([img_x, img_y]),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    selected_frames = np.arange(begin_frame, end_frame, skip_frame).tolist()

    # ?
    
    train_set, valid_set = Dataset_CRNN(os.path.join(data_path, "train"), train_list, train_label, selected_frames, transform=transform), \
                           Dataset_CRNN(os.path.join(data_path, "val"), test_list, test_label, selected_frames, transform=transform)

    train_loader = data.DataLoader(train_set, **params)
    valid_loader = data.DataLoader(valid_set, **params)

    # Create model
    cnn_encoder = EncoderCNN(img_x=img_x, img_y=img_y, fc_hidden1=CNN_fc_hidden1, fc_hidden2=CNN_fc_hidden2,
                             drop_p=dropout_p, CNN_embed_dim=CNN_embed_dim).to(device)

    rnn_decoder = DecoderRNN(CNN_embed_dim=CNN_embed_dim, h_RNN_layers=RNN_hidden_layers, h_RNN=RNN_hidden_nodes, 
                             h_FC_dim=RNN_FC_dim, drop_p=dropout_p, num_classes=k).to(device)

#     # Parallelize model to multiple GPUs
#     if torch.cuda.device_count() > 1:
#         print("Using", torch.cuda.device_count(), "GPUs!")
#         cnn_encoder = nn.DataParallel(cnn_encoder)
#         rnn_decoder = nn.DataParallel(rnn_decoder)

    crnn_params = list(cnn_encoder.parameters()) + list(rnn_decoder.parameters())
    optimizer = torch.optim.Adam(crnn_params, lr=learning_rate)


    # record training process
    epoch_train_losses = []
    epoch_train_scores = []
    epoch_test_losses = []
    epoch_test_scores = []

    # start training
    for epoch in range(epochs):
        # train, test model
        train_losses, train_scores = train(log_interval, [cnn_encoder, rnn_decoder], device, train_loader, optimizer, epoch)
        epoch_test_loss, epoch_test_score = validation([cnn_encoder, rnn_decoder], device, optimizer, valid_loader, epoch)

        # save results
        epoch_train_losses.append(train_losses)
        epoch_train_scores.append(train_scores)
        epoch_test_losses.append(epoch_test_loss)
        epoch_test_scores.append(epoch_test_score)

        # save all train test results
        A = np.array(epoch_train_losses)
        B = np.array(epoch_train_scores)
        C = np.array(epoch_test_losses)
        D = np.array(epoch_test_scores)
        np.save('./CRNN_epoch_training_losses.npy', A)
        np.save('./CRNN_epoch_training_scores.npy', B)
        np.save('./CRNN_epoch_test_loss.npy', C)
        np.save('./CRNN_epoch_test_score.npy', D)

    # plot
    fig = plt.figure(figsize=(10, 4))
    plt.subplot(121)
    plt.plot(np.arange(1, epochs + 1), A[:, -1])  # train loss (on epoch end)
    plt.plot(np.arange(1, epochs + 1), C)         #  test loss (on epoch end)
    plt.title("model loss")
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend(['train', 'test'], loc="upper left")
    # 2nd figure
    plt.subplot(122)
    plt.plot(np.arange(1, epochs + 1), B[:, -1])  # train accuracy (on epoch end)
    plt.plot(np.arange(1, epochs + 1), D)         #  test accuracy (on epoch end)
    plt.title("training scores")
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.legend(['train', 'test'], loc="upper left")
    title = "./fig_sub_Jester_CRNN.png"
    plt.savefig(title, dpi=600)
    # plt.close(fig)
    plt.show()

In [ ]:
if __name__ == "__main__":
    main()

Train Epoch: 1 [300/11291 (3%)]	Loss: 2.299834, Accu: 13.33%
Train Epoch: 1 [600/11291 (5%)]	Loss: 2.300450, Accu: 3.33%
Train Epoch: 1 [900/11291 (8%)]	Loss: 2.306102, Accu: 6.67%
Train Epoch: 1 [1200/11291 (11%)]	Loss: 2.299658, Accu: 3.33%
Train Epoch: 1 [1500/11291 (13%)]	Loss: 2.306126, Accu: 13.33%
Train Epoch: 1 [1800/11291 (16%)]	Loss: 2.305005, Accu: 3.33%
Train Epoch: 1 [2100/11291 (19%)]	Loss: 2.298254, Accu: 16.67%
Train Epoch: 1 [2400/11291 (21%)]	Loss: 2.295262, Accu: 6.67%
Train Epoch: 1 [2700/11291 (24%)]	Loss: 2.303683, Accu: 13.33%
Train Epoch: 1 [3000/11291 (27%)]	Loss: 2.301844, Accu: 6.67%
Train Epoch: 1 [3300/11291 (29%)]	Loss: 2.303556, Accu: 3.33%
Train Epoch: 1 [3600/11291 (32%)]	Loss: 2.294472, Accu: 13.33%
Train Epoch: 1 [3900/11291 (34%)]	Loss: 2.306377, Accu: 0.00%
Train Epoch: 1 [4200/11291 (37%)]	Loss: 2.288962, Accu: 16.67%
Train Epoch: 1 [4500/11291 (40%)]	Loss: 2.302506, Accu: 10.00%
Train Epoch: 1 [4800/11291 (42%)]	Loss: 2.301205, Accu: 0.00%
Train E

Train Epoch: 4 [5100/11291 (45%)]	Loss: 2.238179, Accu: 20.00%
Train Epoch: 4 [5400/11291 (48%)]	Loss: 2.346227, Accu: 13.33%
Train Epoch: 4 [5700/11291 (50%)]	Loss: 2.259326, Accu: 20.00%
Train Epoch: 4 [6000/11291 (53%)]	Loss: 2.386608, Accu: 6.67%
Train Epoch: 4 [6300/11291 (56%)]	Loss: 2.290549, Accu: 10.00%
Train Epoch: 4 [6600/11291 (58%)]	Loss: 2.217415, Accu: 33.33%
Train Epoch: 4 [6900/11291 (61%)]	Loss: 2.234910, Accu: 20.00%
Train Epoch: 4 [7200/11291 (64%)]	Loss: 2.297312, Accu: 13.33%
Train Epoch: 4 [7500/11291 (66%)]	Loss: 2.242831, Accu: 10.00%
Train Epoch: 4 [7800/11291 (69%)]	Loss: 2.226393, Accu: 26.67%
Train Epoch: 4 [8100/11291 (72%)]	Loss: 2.269879, Accu: 6.67%
Train Epoch: 4 [8400/11291 (74%)]	Loss: 2.219188, Accu: 23.33%
Train Epoch: 4 [8700/11291 (77%)]	Loss: 2.294271, Accu: 10.00%
Train Epoch: 4 [9000/11291 (80%)]	Loss: 2.227433, Accu: 20.00%
Train Epoch: 4 [9300/11291 (82%)]	Loss: 2.279032, Accu: 13.33%
Train Epoch: 4 [9600/11291 (85%)]	Loss: 2.231400, Accu: 1

Train Epoch: 7 [9900/11291 (88%)]	Loss: 2.186033, Accu: 30.00%
Train Epoch: 7 [10200/11291 (90%)]	Loss: 1.952615, Accu: 30.00%
Train Epoch: 7 [10500/11291 (93%)]	Loss: 2.103530, Accu: 23.33%
Train Epoch: 7 [10800/11291 (95%)]	Loss: 1.998035, Accu: 23.33%
Train Epoch: 7 [11100/11291 (98%)]	Loss: 1.975627, Accu: 30.00%

Test set (1411 samples): Average loss: 2.1095, Accuracy: 24.24%

Epoch 7 model saved!
Train Epoch: 8 [300/11291 (3%)]	Loss: 1.999081, Accu: 26.67%
Train Epoch: 8 [600/11291 (5%)]	Loss: 1.983601, Accu: 33.33%
Train Epoch: 8 [900/11291 (8%)]	Loss: 1.885367, Accu: 33.33%
Train Epoch: 8 [1200/11291 (11%)]	Loss: 1.997138, Accu: 23.33%
Train Epoch: 8 [1500/11291 (13%)]	Loss: 1.861692, Accu: 43.33%
Train Epoch: 8 [1800/11291 (16%)]	Loss: 2.015401, Accu: 30.00%
Train Epoch: 8 [2100/11291 (19%)]	Loss: 2.211111, Accu: 20.00%
Train Epoch: 8 [2400/11291 (21%)]	Loss: 1.955205, Accu: 36.67%
Train Epoch: 8 [2700/11291 (24%)]	Loss: 2.234811, Accu: 20.00%
Train Epoch: 8 [3000/11291 (27%)]

Train Epoch: 11 [2700/11291 (24%)]	Loss: 1.670300, Accu: 40.00%
Train Epoch: 11 [3000/11291 (27%)]	Loss: 1.597772, Accu: 46.67%
Train Epoch: 11 [3300/11291 (29%)]	Loss: 1.634016, Accu: 40.00%
Train Epoch: 11 [3600/11291 (32%)]	Loss: 1.595057, Accu: 53.33%
Train Epoch: 11 [3900/11291 (34%)]	Loss: 1.973713, Accu: 26.67%
Train Epoch: 11 [4200/11291 (37%)]	Loss: 1.646212, Accu: 46.67%
Train Epoch: 11 [4500/11291 (40%)]	Loss: 1.973778, Accu: 23.33%
Train Epoch: 11 [4800/11291 (42%)]	Loss: 1.498554, Accu: 43.33%
Train Epoch: 11 [5100/11291 (45%)]	Loss: 1.754393, Accu: 36.67%
Train Epoch: 11 [5400/11291 (48%)]	Loss: 1.795821, Accu: 30.00%
Train Epoch: 11 [5700/11291 (50%)]	Loss: 1.655784, Accu: 36.67%
Train Epoch: 11 [6000/11291 (53%)]	Loss: 1.905223, Accu: 33.33%
Train Epoch: 11 [6300/11291 (56%)]	Loss: 2.196051, Accu: 20.00%
Train Epoch: 11 [6600/11291 (58%)]	Loss: 1.878654, Accu: 33.33%
Train Epoch: 11 [6900/11291 (61%)]	Loss: 1.864751, Accu: 43.33%
Train Epoch: 11 [7200/11291 (64%)]	Loss:

Train Epoch: 14 [6600/11291 (58%)]	Loss: 1.618941, Accu: 43.33%
Train Epoch: 14 [6900/11291 (61%)]	Loss: 1.849803, Accu: 30.00%
Train Epoch: 14 [7200/11291 (64%)]	Loss: 1.519549, Accu: 50.00%
Train Epoch: 14 [7500/11291 (66%)]	Loss: 1.426838, Accu: 50.00%
Train Epoch: 14 [7800/11291 (69%)]	Loss: 1.504105, Accu: 43.33%
Train Epoch: 14 [8100/11291 (72%)]	Loss: 1.861728, Accu: 36.67%
Train Epoch: 14 [8400/11291 (74%)]	Loss: 1.244117, Accu: 50.00%
Train Epoch: 14 [8700/11291 (77%)]	Loss: 1.518197, Accu: 40.00%
Train Epoch: 14 [9000/11291 (80%)]	Loss: 1.307816, Accu: 56.67%
Train Epoch: 14 [9300/11291 (82%)]	Loss: 1.318134, Accu: 63.33%
Train Epoch: 14 [9600/11291 (85%)]	Loss: 1.384173, Accu: 46.67%
Train Epoch: 14 [9900/11291 (88%)]	Loss: 1.414358, Accu: 46.67%
Train Epoch: 14 [10200/11291 (90%)]	Loss: 1.502051, Accu: 50.00%
Train Epoch: 14 [10500/11291 (93%)]	Loss: 1.579587, Accu: 26.67%
Train Epoch: 14 [10800/11291 (95%)]	Loss: 1.487026, Accu: 43.33%
Train Epoch: 14 [11100/11291 (98%)]	L

Train Epoch: 17 [10500/11291 (93%)]	Loss: 1.002819, Accu: 63.33%
Train Epoch: 17 [10800/11291 (95%)]	Loss: 0.959736, Accu: 70.00%
Train Epoch: 17 [11100/11291 (98%)]	Loss: 1.178431, Accu: 60.00%

Test set (1411 samples): Average loss: 2.4636, Accuracy: 27.85%

Epoch 17 model saved!
Train Epoch: 18 [300/11291 (3%)]	Loss: 1.163982, Accu: 53.33%
Train Epoch: 18 [600/11291 (5%)]	Loss: 1.143237, Accu: 60.00%
Train Epoch: 18 [900/11291 (8%)]	Loss: 1.059573, Accu: 66.67%
Train Epoch: 18 [1200/11291 (11%)]	Loss: 1.118328, Accu: 56.67%
Train Epoch: 18 [1500/11291 (13%)]	Loss: 1.222998, Accu: 63.33%
Train Epoch: 18 [1800/11291 (16%)]	Loss: 0.935010, Accu: 73.33%
Train Epoch: 18 [2100/11291 (19%)]	Loss: 0.803184, Accu: 80.00%
Train Epoch: 18 [2400/11291 (21%)]	Loss: 1.057136, Accu: 60.00%
Train Epoch: 18 [2700/11291 (24%)]	Loss: 0.861009, Accu: 73.33%
Train Epoch: 18 [3000/11291 (27%)]	Loss: 1.212163, Accu: 53.33%
Train Epoch: 18 [3300/11291 (29%)]	Loss: 0.985832, Accu: 63.33%
Train Epoch: 18 [36

Train Epoch: 21 [3000/11291 (27%)]	Loss: 0.990507, Accu: 56.67%
Train Epoch: 21 [3300/11291 (29%)]	Loss: 0.981368, Accu: 76.67%
Train Epoch: 21 [3600/11291 (32%)]	Loss: 0.865385, Accu: 66.67%
Train Epoch: 21 [3900/11291 (34%)]	Loss: 0.788811, Accu: 70.00%
Train Epoch: 21 [4200/11291 (37%)]	Loss: 0.787010, Accu: 73.33%
Train Epoch: 21 [4500/11291 (40%)]	Loss: 0.893853, Accu: 70.00%
Train Epoch: 21 [4800/11291 (42%)]	Loss: 0.988241, Accu: 56.67%
Train Epoch: 21 [5100/11291 (45%)]	Loss: 0.639623, Accu: 76.67%
Train Epoch: 21 [5400/11291 (48%)]	Loss: 1.236294, Accu: 43.33%
Train Epoch: 21 [5700/11291 (50%)]	Loss: 0.885597, Accu: 70.00%
Train Epoch: 21 [6000/11291 (53%)]	Loss: 0.782795, Accu: 76.67%
Train Epoch: 21 [6300/11291 (56%)]	Loss: 0.596961, Accu: 80.00%
Train Epoch: 21 [6600/11291 (58%)]	Loss: 0.925026, Accu: 63.33%
Train Epoch: 21 [6900/11291 (61%)]	Loss: 0.901477, Accu: 66.67%
Train Epoch: 21 [7200/11291 (64%)]	Loss: 0.703064, Accu: 73.33%
Train Epoch: 21 [7500/11291 (66%)]	Loss: